In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install ultralytics

In [3]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [4]:
train_path_img = "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/images/train/"
train_path_label = "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/labels/train/"
val_path_img = "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/images/val/"
val_path_label = "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/labels/val/"
test_path = "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/test"

In [5]:
!ls '/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/data'

22d_f_120_6ty.jpg  22d_f_85_fr.txt    6ty_f_90_we.jpg	 fre_f_95_54r.txt   rt3_f_20_54r.jpg
22d_f_120_6ty.txt  22d_f_90.jpg       6ty_f_90_we.txt	 fr_f_100_we.jpg    rt3_f_20_54r.txt
22d_f_125_fre.jpg  22d_f_90.txt       fre_f_10_54r.jpg	 fr_f_100_we.txt    rt3_f_25_6ty.jpg
22d_f_125_fre.txt  54r_f_120.jpg      fre_f_10_54r.txt	 fr_f_105_6ty.jpg   rt3_f_25_6ty.txt
22d_f_125.jpg	   54r_f_120.txt      fre_f_115.jpg	 fr_f_105_6ty.txt   rt3_f_35_rt3.jpg
22d_f_125.txt	   54r_f_5_22d.jpg    fre_f_115.txt	 fr_f_10.jpg	    rt3_f_35_rt3.txt
22d_f_135_we.jpg   54r_f_5_22d.txt    fre_f_145_6ty.jpg  fr_f_10.txt	    rt3_f_95.jpg
22d_f_135_we.txt   54r_f_65.jpg       fre_f_145_6ty.txt  fr_f_130.jpg	    rt3_f_95.txt
22d_f_15_rt3.jpg   54r_f_65.txt       fre_f_155_we.jpg	 fr_f_130.txt	    we_f_100.jpg
22d_f_15_rt3.txt   54r_f_75.jpg       fre_f_155_we.txt	 fr_f_140_fre.jpg   we_f_100.txt
22d_f_165_fr.jpg   54r_f_75.txt       fre_f_190_22d.jpg  fr_f_140_fre.txt   we_f_105.jpg
22d_f_165_fr.txt   54r_f_75

In [6]:

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
#train_test_split('/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/data') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images
train_test_split('/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/data/')

------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


In [7]:
#creating the .yaml file for the paths

!touch /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/config.yaml

In [8]:
from ultralytics import YOLO
import os

model=YOLO('yolov8m.pt')
result=model.train(data='/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/config.yaml', epochs=5, imgsz=640)



Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/config.yaml, epochs=5, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

train: Scanning /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/labels/train.cache... 66 images, 0 backgrounds, 0 corrupt: 100%|██████████| 66/66 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/yolo_data/labels/train.cache... 66 images, 0 backgrounds, 0 corrupt: 100%|██████████| 66/66 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        5/5      7.19G      1.477     0.7562     0.9849         52        640: 100%|██████████| 5/5 [00:03<00:00,  1.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.00it/s]

                   all         66        750      0.801      0.737      0.759      0.421



5 epochs completed in 0.024 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.28s/it]


                   all         66        750      0.803      0.737      0.759      0.421
                player         66        701      0.907      0.984      0.985      0.624
              football         66         49      0.699       0.49      0.533      0.217
Speed: 0.2ms preprocess, 10.7ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs/detect/train2


In [9]:
#testing the model with best.pt model and with a new image

infer1=YOLO('/content/runs/detect/train/weights/best.pt')

infer1.predict('/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/fr_f_130 copy.jpg', save=True ,save_txt=True)


image 1/1 /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/fr_f_130 copy.jpg: 384x640 5 players, 117.4ms
Speed: 2.7ms preprocess, 117.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'player', 1: 'football'}
 obb: None
 orig_img: array([[[110, 125, 121],
         [110, 125, 121],
         [110, 125, 121],
         ...,
         [155, 160, 158],
         [155, 160, 158],
         [155, 160, 158]],
 
        [[110, 125, 121],
         [110, 125, 121],
         [110, 125, 121],
         ...,
         [155, 160, 158],
         [155, 160, 158],
         [155, 160, 158]],
 
        [[110, 125, 121],
         [110, 125, 121],
         [110, 125, 121],
         ...,
         [155, 160, 158],
         [155, 160, 158],
         [155, 160, 158]],
 
        ...,
 
        [[ 26,  59,  44],
         [ 25,  58,  43],
         [ 25,  58,  43],
         ...,
         [ 22,  55,  41],
         [ 21,  54,  40],
         [ 20,  53,  39]],
 
        [[ 25,  58,  43],
         [ 25,  58,  43],
         [ 26,  60,  43],
         ...,
   

In [10]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [11]:
infer=YOLO('/content/runs/detect/train/weights/best.pt')

infer.predict('/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/test', save=True, save_txt=True)


image 1/4 /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/test/Football_in_Bloomington,_Indiana,_1995.jpg: 448x640 7 players, 137.0ms
image 2/4 /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/test/fr_f_130 copy.jpg: 384x640 5 players, 29.0ms
image 3/4 /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/test/fr_f_140_fre copy.jpg: 384x640 13 players, 25.4ms
image 4/4 /content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/test/wynton.jpg: 448x640 10 players, 28.9ms
Speed: 3.8ms preprocess, 55.1ms inference, 2.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict4
4 labels saved to runs/detect/predict4/labels


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'player', 1: 'football'}
 obb: None
 orig_img: array([[[ 24,  32,  45],
         [ 24,  27,  41],
         [ 25,  31,  44],
         ...,
         [ 73,  80,  89],
         [ 75,  85,  92],
         [ 63,  73,  80]],
 
        [[ 27,  33,  46],
         [ 29,  32,  46],
         [ 29,  32,  46],
         ...,
         [ 70,  79,  88],
         [ 69,  78,  87],
         [ 72,  81,  90]],
 
        [[ 32,  35,  49],
         [ 34,  37,  51],
         [ 33,  36,  50],
         ...,
         [ 69,  78,  87],
         [ 70,  81,  89],
         [ 69,  80,  88]],
 
        ...,
 
        [[ 64,  95,  94],
         [ 64,  95,  94],
         [ 64,  95,  94],
         ...,
         [ 72, 101,  98],
         [ 71, 100,  97],
         [ 70,  99,  96]],
 
        [[ 63,  94,  93],
         [ 63,  94,  93],
         [ 63,  94,  93],
         ...,
   

In [15]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16403, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 16403 (delta 154), reused 161 (delta 77), pack-reused 16106
Receiving objects: 100% (16403/16403), 15.15 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (11187/11187), done.
/content/yolov5


In [16]:
# Run YOLOv8 detection
!python /content/yolov5/detect.py --weights /content/runs/detect/train/weights/best.pt --img-size 640 --source "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4"

# Compress the resulting video using ffmpeg
!ffmpeg -i "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4" -vcodec libx265 "/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/football.mp4"


detect: weights=['/content/runs/detect/train/weights/best.pt'], source=/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'Pillow>=10.0.1'] not found, attempting AutoUpdate...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
"""from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Specify the path to the input video
video_path = '/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4'

# Make predictions on the video
results = model.predict(source=video_path)

# Visualize the predictions
results.show()

# Specify the path to the output video
#output_video_path = 'path_to_output_video.mp4'

# Save the annotated video
#results.save(save_dir=output_video_path)"""


"from ultralytics import YOLO\n\n# Load the trained YOLOv8 model\nmodel = YOLO('/content/runs/detect/train/weights/best.pt')\n\n# Specify the path to the input video\nvideo_path = '/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4'\n\n# Make predictions on the video\nresults = model.predict(source=video_path)\n\n# Visualize the predictions\nresults.show()\n\n# Specify the path to the output video\n#output_video_path = 'path_to_output_video.mp4'\n\n# Save the annotated video\n#results.save(save_dir=output_video_path)"

In [14]:
import torch
from pathlib import Path
from IPython.display import display, HTML

# Load YOLOv8 model from torch hub
model = torch.hub.load('ultralytics/yolov5:v5.0', 'yolov5s', pretrained=True)

# Specify the path to the input video
video_path = '/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/video (2160p).mp4'

# Perform inference on the video
results = model(video_path)

# Save the annotated video in a compatible format
output_video_path = '/content/gdrive/MyDrive/computer_vision_eng/football_people_detection_YOLOV8/football.mov'
results.save(save_dir=output_video_path, save_all=True, codec='mp4v', fps=30)

# Display the annotated video in the notebook
video_html = f"""
<video width="640" height="360" controls>
  <source src="{output_video_path}" type="video/mp4">
</video>
"""
display(HTML(video_html))


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/v5.0" to /root/.cache/torch/hub/v5.0.zip
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF

100%|██████████| 14.1M/14.1M [00:00<00:00, 192MB/s]


Exception: Cache maybe be out of date, try force_reload=True. See https://github.com/ultralytics/yolov5/issues/36 for help.